In [1]:
cd /Users/jz-rolling/Desktop/OMEGA_GUI/notebooks/

/Users/jz-rolling/Desktop/OMEGA_GUI/notebooks


In [36]:
import momia2 as mo
from skimage import measure, feature, filters, transform, morphology
import numpy as np
import pandas as pd
import read_roi as rr
import matplotlib.pyplot as plt
import glob
import pickle as pk
import os 
import tifffile

In [48]:
src = '/Users/jz-rolling/Desktop/Size_calculation/20220118/'

In [49]:
for i in range(8):
    days = []
    dates = []
    images = []
    for f in glob.glob(src+'Projection_redo_20220119/Plate-{}*.tif'.format(i)):
        plate,date,day = f.split('/')[-1].split('.tif')[0].split('_')
        img = plt.imread(f)
        days.append(int(day))
        dates.append(date)
        images.append(img)
    images = np.array(images)
    days = np.array(days)
    dates = np.array(dates)
    days_sorted_arg = np.argsort(days)
    images = images[days_sorted_arg]
    days = days[days_sorted_arg]
    dates = dates[days_sorted_arg]
    corrected,_w = correct_tl(images)
    plate_folder = src+'Alignment_20220120/{}/'.format(plate)
    if not os.path.isdir(plate_folder):
        os.mkdir(plate_folder)
    for j,img in enumerate(corrected):
        img_name = plate_folder+'{}_{}.tif'.format(days[j],dates[j])
        tifffile.imsave(img_name,
                        (min_max(img)*255).astype(np.uint8),imagej=True)

In [35]:
def min_max(data):
    return (data-data.min())/(data.max()-data.min())

def xydrift_correction(target_img, shift, max_drift=25):
    if max(np.abs(shift)) <= max_drift:
        return shift_image(target_img, np.array(shift))
    else:
        return target_img
    
def get_xydrift(ref_img, target_img):
    from skimage import registration
    shift, error, _diff = registration.phase_cross_correlation(ref_img, target_img, upsample_factor=10)
    return shift

def shift_image(img, shift):
    from scipy import ndimage as ndi
    """
    correct xy drift between phase contrast image and fluorescent image(s)
    :param img: input image
    :param shift: subpixel xy drift
    :return: drift corrected image
    """
    offset_image = ndi.fourier_shift(np.fft.fftn(img), shift)
    offset_image = np.fft.ifftn(offset_image)
    offset_image = np.round(offset_image.real)
    return offset_image


def correct_tl(pos_data,max_drift=100):
    is_weird = [0]
    last_drift = np.array([0,0])
    output_data = []
    for t in range(len(pos_data)):
        if t == 0:
            output_data.append(pos_data[t])
        else:
            shift = get_xydrift(pos_data[t-1],pos_data[t])
            if np.abs(shift).max()>max_drift:
                is_weird.append(1)
            else:
                is_weird.append(0)
            last_drift = last_drift + shift
            output_data.append(xydrift_correction(pos_data[t],last_drift,max_drift=max_drift))
    output_data = np.array(output_data)
    return output_data, is_weird